# Waterfilling Levels

In [1]:
import sys
sys.path.insert(1, '../../functions_multi_resource')
import importlib
import numpy as np
import nbformat
import plotly.express
import plotly.express as px
import pandas as pd
from scipy.optimize import minimize
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from food_bank_functions import *
import time
# importlib.reload(food_bank_functions)

In [2]:
np.random.seed(3435)

In [3]:
n = 3
k = 4
size = [1., 2., 3.]
B = np.zeros((k, n*k))
for i in range(n):
    B[:,k*i:k*(i+1)] = size[i]*np.eye(k)
print(B)

[[1. 0. 0. 0. 2. 0. 0. 0. 3. 0. 0. 0.]
 [0. 1. 0. 0. 0. 2. 0. 0. 0. 3. 0. 0.]
 [0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 3. 0.]
 [0. 0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 3.]]


# Generating Distribution

In [4]:
n = 6
k = 9

#### Different Population of Locations

In [5]:
# size = [1, 1, 1, 1, 1, 1]
size = [928, 1200, 420, 429, 103, 393]
size = size / np.sum(size) * 100
county = ['Broome', 'Steuben', 'Chemung', 'Tioga', 'Schuyler', 'Tompkins']
print(county)
print(size)

['Broome', 'Steuben', 'Chemung', 'Tioga', 'Schuyler', 'Tompkins']
[26.72041463 34.55226029 12.0932911  12.35243305  2.96573568 11.31586525]


#### Distribution on Weights

In [43]:
product = ['cereal', 'diapers', 'pasta', 'paper', 'prepared_meals', 'rice', 'meat', 'fruit', 'produce']
w = [3.9, 3.5, 3.2, 3, 2.8, 2.7, 1.9, 1.2, .2]
print(product)
print(w)
budget = np.sum(w)*np.ones(k)
print(budget)

['cereal', 'diapers', 'pasta', 'paper', 'prepared_meals', 'rice', 'meat', 'fruit', 'produce']
[3.9, 3.5, 3.2, 3, 2.8, 2.7, 1.9, 1.2, 0.2]
[22.4 22.4 22.4 22.4 22.4 22.4 22.4 22.4 22.4]


In [44]:
w_1 = [1, 0, 1, 0, 0, 1, 1, 1, 1] # soup kitchen 
w_2 = [1, 1, 1, 1, 1, 1, 1, 1, 1] # general warehouse
w_3 = np.random.randint(0,2,9)
w_4 = np.random.randint(0,2,9)
w_5 = np.random.randint(0,2,9)
w_6 = np.random.randint(0,2,9)
w_7 = np.random.randint(0,2,9)
w_8 = np.random.randint(0,2,9)
weight_matrix = np.asarray([ w_1, w_2, w_3, w_4, w_5, w_6, w_7, w_8])
print(weight_matrix)
weight_distribution = [1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8, 1/8]

[[1 0 1 0 0 1 1 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 0 0 0]
 [0 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 1 1 0 0]
 [0 1 1 1 0 1 1 1 1]
 [0 1 1 0 1 1 1 1 0]
 [1 0 1 0 0 0 1 0 1]]


In [45]:
expected_weights = np.zeros((n,k))

In [46]:
for i in range(n):
    for j in range(k):
#         print(i,j)
        expected_weights[i,j] = w[j] * (1/8) * (w_1[j] + w_2[j] + w_3[j] + w_4[j] + w_5[j] + w_6[j] + w_7[j]+w_8[j])
print(expected_weights)

[[2.4375 2.1875 2.8    1.5    1.05   2.025  1.425  0.6    0.125 ]
 [2.4375 2.1875 2.8    1.5    1.05   2.025  1.425  0.6    0.125 ]
 [2.4375 2.1875 2.8    1.5    1.05   2.025  1.425  0.6    0.125 ]
 [2.4375 2.1875 2.8    1.5    1.05   2.025  1.425  0.6    0.125 ]
 [2.4375 2.1875 2.8    1.5    1.05   2.025  1.425  0.6    0.125 ]
 [2.4375 2.1875 2.8    1.5    1.05   2.025  1.425  0.6    0.125 ]]


In [47]:
size

array([26.72041463, 34.55226029, 12.0932911 , 12.35243305,  2.96573568,
       11.31586525])

In [48]:
x, sol = solve(expected_weights, n, k, budget, size)

In [49]:
x = np.reshape(x, (n,k))

In [50]:
print(x)

[[0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]
 [0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224 0.224]]


In [51]:
print(excess(x, budget, size))
print(envy_utility(x, expected_weights))
print(proportionality_utility(x, expected_weights, size, budget))

[1.18423789e-15 1.18423789e-15 1.18423789e-15 1.18423789e-15
 1.18423789e-15 1.18423789e-15 1.18423789e-15 1.18423789e-15
 1.18423789e-15]
[0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [52]:
x.shape

(6, 9)

In [53]:
print(n)

6


### Test

In [54]:
wm = np.zeros((8,k))
for i in range(8):
    wm[i,:] = np.multiply(weight_matrix[i,:], w)
print(wm)

[[3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 3.  2.8 2.7 0.  0.  0. ]
 [0.  3.5 0.  0.  0.  0.  0.  0.  0.2]
 [3.9 3.5 3.2 3.  0.  2.7 1.9 0.  0. ]
 [0.  3.5 3.2 3.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 0.  2.8 2.7 1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  0.  1.9 0.  0.2]]


In [55]:
obs_types = np.random.randint(0,8,n)
print(obs_types)

observed_weights = np.zeros((n, k))
for i in range(n):
    observed_weights[i,:] = np.multiply(weight_matrix[obs_types[i], :], w)
print(observed_weights)

[1 1 6 7 5 0]
[[3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 0.  2.8 2.7 1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  0.  1.9 0.  0.2]
 [0.  3.5 3.2 3.  0.  2.7 1.9 1.2 0.2]
 [3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]]


In [56]:
opt, _ = solve(observed_weights, n, k, budget, size)
opt = np.reshape(opt, (n,k))
print(np.around(opt, decimals=4))

[[0.2639 0.2086 0.222  0.2864 0.2131 0.1826 0.1726 0.1888 0.2236]
 [0.1369 0.2723 0.1056 0.3854 0.3008 0.1921 0.1886 0.2264 0.2289]
 [0.     0.5206 0.1894 0.     0.5219 0.21   0.1487 0.2325 0.    ]
 [0.5736 0.     0.5495 0.     0.     0.     0.5011 0.     0.3495]
 [0.     0.378  0.2308 0.4831 0.     0.2695 0.2134 0.2711 0.2408]
 [0.3124 0.     0.2703 0.     0.     0.6667 0.2341 0.5231 0.3079]]


In [57]:
for i in range(n):
    print(np.around([observed_weights[i,j] / (np.dot(opt[i,:], observed_weights[i,:])) for j in range(k)], decimals=2))

[0.78 0.7  0.64 0.6  0.56 0.54 0.38 0.24 0.04]
[0.78 0.7  0.64 0.6  0.56 0.54 0.38 0.24 0.04]
[0.   0.7  0.64 0.   0.56 0.54 0.38 0.24 0.  ]
[0.78 0.   0.64 0.   0.   0.   0.38 0.   0.04]
[0.   0.7  0.64 0.6  0.   0.54 0.38 0.24 0.04]
[0.78 0.   0.64 0.   0.   0.54 0.38 0.24 0.04]


In [58]:
## NEED TO GET RID OF THE 0S IN THE DISTRIBUTION.

In [59]:
num_types = np.zeros(len(weight_distribution))
size_factors = np.zeros(len(weight_distribution))
for i in range(n):
    num_types[obs_types[i]] += 1
    size_factors[obs_types[i]] += size[i]
print(num_types)
print(size_factors)

[1. 2. 0. 0. 0. 1. 1. 1.]
[11.31586525 61.27267492  0.          0.          0.          2.96573568
 12.0932911  12.35243305]


In [60]:
print(obs_types)
print(num_types)
print(size_factors)

[1 1 6 7 5 0]
[1. 2. 0. 0. 0. 1. 1. 1.]
[11.31586525 61.27267492  0.          0.          0.          2.96573568
 12.0932911  12.35243305]


In [61]:
num_types[num_types != 0]

array([1., 2., 1., 1., 1.])

In [62]:
size_factors[num_types != 0]

array([11.31586525, 61.27267492,  2.96573568, 12.0932911 , 12.35243305])

In [63]:
wm[num_types != 0,:]

array([[3.9, 0. , 3.2, 0. , 0. , 2.7, 1.9, 1.2, 0.2],
       [3.9, 3.5, 3.2, 3. , 2.8, 2.7, 1.9, 1.2, 0.2],
       [0. , 3.5, 3.2, 3. , 0. , 2.7, 1.9, 1.2, 0.2],
       [0. , 3.5, 3.2, 0. , 2.8, 2.7, 1.9, 1.2, 0. ],
       [3.9, 0. , 3.2, 0. , 0. , 0. , 1.9, 0. , 0.2]])

In [64]:
print(wm[num_types != 0], size_factors[num_types != 0])

[[3.9 0.  3.2 0.  0.  2.7 1.9 1.2 0.2]
 [3.9 3.5 3.2 3.  2.8 2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 3.  0.  2.7 1.9 1.2 0.2]
 [0.  3.5 3.2 0.  2.8 2.7 1.9 1.2 0. ]
 [3.9 0.  3.2 0.  0.  0.  1.9 0.  0.2]] [11.31586525 61.27267492  2.96573568 12.0932911  12.35243305]


In [65]:
opt_2, _ = solve(wm[num_types != 0,:], len(wm[num_types != 0,:]), k, budget, size_factors[num_types != 0])
opt_2 = np.reshape(opt_2, (len(wm[num_types != 0,:]),k))
print(np.around(opt_2, decimals=4))

[[0.3209 0.     0.2869 0.     0.     0.6401 0.2442 0.4962 0.2992]
 [0.1951 0.2638 0.1222 0.3433 0.2945 0.1753 0.1555 0.2287 0.2235]
 [0.     0.3727 0.2496 0.461  0.     0.2746 0.2205 0.2694 0.2394]
 [0.     0.4245 0.3475 0.     0.3602 0.2978 0.2167 0.1633 0.    ]
 [0.5514 0.     0.5442 0.     0.     0.     0.5532 0.     0.3732]]


In [66]:
new_wm = wm[num_types != 0, :]

In [69]:
print(excess(opt_2, budget, size_factors[num_types != 0]))
print(envy_utility(opt_2, wm[num_types != 0,:]))
print(proportionality_utility(opt_2, wm[num_types != 0,:], size_factors[num_types != 0], budget))

[-1.71011294e-10 -1.55387880e-10 -1.48245505e-10 -1.28083855e-10
 -1.22783916e-10 -1.22607702e-10 -8.78266349e-11 -5.42961232e-11
 -8.85833629e-12]
[ 2.57123747e-04  2.06980244e-04 -1.00843540e+00 -1.31167637e+00
 -2.32408350e+00]
[-2.08312266e+00  2.72007630e-05 -1.50070427e+00 -1.59045004e+00
 -2.95697978e+00]


In [70]:
arr = np.arange(0, len(weight_matrix))
used_types = arr[num_types != 0]
alloc = np.zeros((n,k))
for i in range(n):
    new_index = np.argmin(np.abs(used_types - obs_types[i]))
    alloc[i,:] = opt_2[new_index,:]

In [71]:
obs_types

array([1, 1, 6, 7, 5, 0])

In [72]:
print(np.around(alloc, decimals=4))

[[0.1951 0.2638 0.1222 0.3433 0.2945 0.1753 0.1555 0.2287 0.2235]
 [0.1951 0.2638 0.1222 0.3433 0.2945 0.1753 0.1555 0.2287 0.2235]
 [0.     0.4245 0.3475 0.     0.3602 0.2978 0.2167 0.1633 0.    ]
 [0.5514 0.     0.5442 0.     0.     0.     0.5532 0.     0.3732]
 [0.     0.3727 0.2496 0.461  0.     0.2746 0.2205 0.2694 0.2394]
 [0.3209 0.     0.2869 0.     0.     0.6401 0.2442 0.4962 0.2992]]


In [33]:
print([np.dot(observed_weights[2,:], alloc[i,:]) for i in range(n)])

[4.592000000000001, 4.592000000000001, 4.592000000000001, 4.592000000000001, 4.592000000000001, 4.592000000000001]


In [73]:
print(np.around(opt, decimals=4))

[[0.2639 0.2086 0.222  0.2864 0.2131 0.1826 0.1726 0.1888 0.2236]
 [0.1369 0.2723 0.1056 0.3854 0.3008 0.1921 0.1886 0.2264 0.2289]
 [0.     0.5206 0.1894 0.     0.5219 0.21   0.1487 0.2325 0.    ]
 [0.5736 0.     0.5495 0.     0.     0.     0.5011 0.     0.3495]
 [0.     0.378  0.2308 0.4831 0.     0.2695 0.2134 0.2711 0.2408]
 [0.3124 0.     0.2703 0.     0.     0.6667 0.2341 0.5231 0.3079]]


In [35]:
[np.dot(opt_2[1,:], new_wm[i,:]) for i in range(len(new_wm))]

[2.9344000000000006, 4.592000000000001, 1.9712000000000003, 0.0, 0.0]

In [36]:
[np.dot(alloc[2,:], observed_weights[i,:]) for i in range(n)]

[2.3296000000000006,
 1.9712000000000003,
 4.592000000000001,
 3.2032000000000007,
 3.2032000000000007,
 2.9344000000000006]

In [37]:
alloc[2,:]

array([0.224, 0.224, 0.224, 0.224, 0.224, 0.224, 0.224, 0.224, 0.224])

In [38]:
obs_types[2]

3

In [74]:
print(excess(alloc, budget, size))
print(envy_utility(alloc, observed_weights))
print(proportionality_utility(alloc, observed_weights, size, budget))

[-1.42509412e-10 -1.29489900e-10 -1.23538513e-10 -1.06737138e-10
 -1.02319930e-10 -1.02173085e-10 -7.31888624e-11 -4.52461772e-11
 -7.38194691e-12]
[ 2.06980244e-04  2.06980244e-04 -1.31167637e+00 -2.32408350e+00
 -1.00843540e+00  2.57123747e-04]
[ 2.72007630e-05  2.72007630e-05 -1.59045004e+00 -2.95697978e+00
 -1.50070427e+00 -2.08312266e+00]


In [75]:
print(excess(opt, budget, size))
print(envy_utility(opt, observed_weights))
print(proportionality_utility(opt, observed_weights, size, budget))

[-8.94265402e-11 -7.96192741e-11 -7.86292513e-11 -6.31891576e-11
 -6.30328382e-11 -6.32217242e-11 -4.68389771e-11 -2.81588086e-11
 -4.59602726e-12]
[ 2.49920609e-08  1.37192473e-05 -1.27994272e+00 -2.42799149e+00
 -1.21834864e+00  1.04565482e-04]
[-2.19339729e-05 -8.23971766e-06 -1.59042196e+00 -2.95680032e+00
 -1.50081325e+00 -2.08309576e+00]
